In [1]:
%matplotlib inline
import torch
from IPython import display
from matplotlib import pyplot as plt
import numpy as np
import random


num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = torch.tensor(np.random.normal(0, 1, (num_examples,num_inputs)), dtype=torch.float)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01,size=labels.size()), dtype=torch.float)

In [2]:
import torch.utils.data as Data

batch_size = 10
# 将训练数据的特征和标签组合
dataset = Data.TensorDataset(features, labels)
# 随机读取⼩批量
data_iter = Data.DataLoader(dataset, batch_size, shuffle=True)

In [3]:
# 这里 data_iter 的使用跟上一节中的一样。让我们读取并打印第一个小批量数据样本。
for X, y in data_iter:
    print(X, y)
    break

tensor([[-0.2428, -0.4984],
        [-0.2276, -1.5541],
        [ 0.0768, -0.8854],
        [ 0.7288, -0.2449],
        [ 0.3102,  0.7145],
        [ 0.0090, -1.8293],
        [-0.8060,  0.2636],
        [ 0.6561,  0.5506],
        [ 0.9565, -0.4212],
        [ 0.3218,  0.7022]]) tensor([ 5.3932,  9.0260,  7.3730,  6.4926,  2.4042, 10.4331,  1.7022,  3.6309,
         7.5474,  2.4709])


In [4]:
# 在实际使用中，最常见的做法是继承 nn.Module ，撰写自⼰的网络/层。一个 nn.Module 实例应该包含一些层以及返回输出的前向传播（forward）方法。下面先
# 来看看如何用 nn.Module 实现一个线性回归模型。
import torch.nn as nn
class LinearNet(nn.Module):
    def __init__(self, n_feature):
        super(LinearNet, self).__init__()
        self.linear = nn.Linear(n_feature, 1)
    # forward 定义前向传播
    def forward(self, x):
        y = self.linear(x)
        return y
    
net = LinearNet(num_inputs)
print(net) # 使用print可以打印出网络的结构

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


In [5]:
net = nn.Sequential(nn.Linear(num_inputs, 1))# 此处还可以传入其他层
print(net)
print(net[0])

Sequential(
  (0): Linear(in_features=2, out_features=1, bias=True)
)
Linear(in_features=2, out_features=1, bias=True)


In [6]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[-0.1648, -0.1939]], requires_grad=True)
Parameter containing:
tensor([0.3040], requires_grad=True)


In [7]:
from torch.nn import init
init.normal_(net[0].weight, mean=0, std=0.01)
init.constant_(net[0].bias, val=0) # 也可以直接修改bias的data:net[0].bias.data.fill_(0)

tensor([0.])

In [8]:
loss = nn.MSELoss()

In [9]:
import torch.optim as optim
optimizer = optim.SGD(net.parameters(), lr=0.03)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [10]:
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        output = net(X)
        l = loss(output, y.view(-1, 1))
        optimizer.zero_grad() # 梯度清零，等价于net.zero_grad()
        l.backward()
        optimizer.step()
    print('迭代次数epoch %d, 损失函数loss: %f' % (epoch, l.item()))

迭代次数epoch 1, 损失函数loss: 0.000259
迭代次数epoch 2, 损失函数loss: 0.000077
迭代次数epoch 3, 损失函数loss: 0.000062


In [11]:
dense = net[0]
print(true_w, dense.weight)
print(true_b, dense.bias)

[2, -3.4] Parameter containing:
tensor([[ 1.9994, -3.4000]], requires_grad=True)
4.2 Parameter containing:
tensor([4.1999], requires_grad=True)
